In [1]:
import findspark
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION'] = spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless - qq > /dev/null
!wget - q http: // www.apache.org/dist/spark /$SPARK_VERSION /$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install - q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
findspark.init()


Get:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:13 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https: // jdbc.postgresql.org/download/postgresql-42.2.16.jar


--2022-04-24 18:20:35--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.69MB/s    in 0.6s    

2022-04-24 18:20:36 (1.69 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config(
    "spark.driver.extraClassPath", "/content/postgresql-42.2.16.jar").getOrCreate()


In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Tools_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""),
                                                sep="\t", header=True, inferSchema=True)
df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   15785389|R2UM5QMHBHC90Q|B00H5U9ZD6|     115362950|WallPeg 12 sq ft ...|           Tools|          5|            0|          0|   N|                Y|Great organizer, ...|Very nice. Will o...| 2015-08-31|
|         US|   47910848|  RF0D1LEIF6L7|B001TJGCS0|     570955425|Nite Ize Nite Daw...|           Tools|          4|    

In [10]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes",
                    "total_votes", "vine", "verified_purchase"])
#vine_df = vine_df.astype({'star_rating':'int'})
vine_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2UM5QMHBHC90Q|          5|            0|          0|   N|                Y|
|  RF0D1LEIF6L7|          4|            0|          0|   N|                Y|
|  RM6YKIWQVNSY|          1|            6|          6|   N|                Y|
|R1RL3L68ASPS36|          4|            0|          0|   N|                Y|
|R1U4XFBFAG34CY|          5|            0|          0|   N|                Y|
|R3KFIK8P0I91PL|          5|            0|          0|   N|                Y|
| RENOAY76PPK1O|          5|            0|          0|   N|                Y|
| RINV884I0NL5V|          1|            0|          0|   N|                Y|
| R5KJH6CXZH2PX|          5|            0|          0|   N|                Y|
| RO69JF6QWD0W1|          5|            0|          0|   N|     

In [11]:
vine_df = vine_df.filter("total_votes<20")
vine_df.show(5)


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2UM5QMHBHC90Q|          5|            0|          0|   N|                Y|
|  RF0D1LEIF6L7|          4|            0|          0|   N|                Y|
|  RM6YKIWQVNSY|          1|            6|          6|   N|                Y|
|R1RL3L68ASPS36|          4|            0|          0|   N|                Y|
|R1U4XFBFAG34CY|          5|            0|          0|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [15]:
vine_helpful_df = vine_df.filter("helpful_votes/total_votes>=.5")
vine_helpful_df.show(5)


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|  RM6YKIWQVNSY|          1|            6|          6|   N|                Y|
|R3L9NQBH3FI55C|          4|            1|          1|   N|                Y|
| RV3KWQBTNIO62|          5|            1|          1|   N|                Y|
|R1M7YUNLZI0G9F|          5|            1|          1|   N|                Y|
| R710G45MKODY9|          2|            1|          1|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [16]:
vine_paid_df = vine_helpful_df.filter("vine == 'Y'")
vine_paid_df.show(5)


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R30KLRGDB7Y7MO|          5|            1|          1|   Y|                N|
|R3FH73XQLEK8KA|          5|            1|          1|   Y|                N|
|R1MTDISLSTFBWP|          4|            2|          3|   Y|                N|
|R1SPEIG6NXUZXF|          5|            1|          1|   Y|                N|
|R20HFNJYEBKDV2|          5|            3|          6|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [17]:
vine_unpaid_df = vine_helpful_df.filter("vine == 'N'")
vine_unpaid_df.show(5)


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|  RM6YKIWQVNSY|          1|            6|          6|   N|                Y|
|R3L9NQBH3FI55C|          4|            1|          1|   N|                Y|
| RV3KWQBTNIO62|          5|            1|          1|   N|                Y|
|R1M7YUNLZI0G9F|          5|            1|          1|   N|                Y|
| R710G45MKODY9|          2|            1|          1|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [21]:
All_reviews = df.count()
print(All_reviews)


1741100


In [25]:
Five_star_review = df.filter("star_rating == '5'").count()
print(Five_star_review)


1113563


In [30]:
Vine_reviews = vine_paid_df.count()
print(Vine_reviews)


3184


In [31]:
Non_Vine_reviews = vine_unpaid_df.count()
print(Non_Vine_reviews)


529880


In [32]:
Vine_reviews_5star = vine_paid_df.filter("star_rating == '5'").count()
Non_Vine_reviews_5star = vine_unpaid_df.filter("star_rating == '5'").count()
print(Vine_reviews_5star)
print(Non_Vine_reviews_5star)


1839
304742


In [28]:
Five_star_paid_review = (vine_paid_df.filter(
    "star_rating == '5'").count()/vine_paid_df.count())*100
print(Five_star_paid_review)


57.757537688442206


In [29]:
Five_star_unpaid_review = (vine_unpaid_df.filter(
    "star_rating == '5'").count()/vine_unpaid_df.count())*100
print(Five_star_unpaid_review)


57.51151204046199
